## AUTOMATIZACION TABLA_PERIODOS BALANCES DE BANCOS

In [27]:
from utils.functions_ia import *
from gspread_formatting  import *

#me me conecto a la spreadsheet y agarro sus datos
key = "1Ed0M09j0yk2LCLU0sBmAUximavxcZoesNmDd1w25TC8"
sheet_name = "PERIODOS"
gc = connect_spreadsheets()
sh = gc.open_by_key(key)
worksheet = sh.worksheet(sheet_name)

wks_data = worksheet.get_all_values() #es una lista de listas de cada fila del spreadsheet

#me conecto a la base de datos
cnxn = connect_database() #genero la conexion con la db

#inserto los datos de la spreadsheet en la db
cursor = cnxn.cursor()

fmt = cellFormat(backgroundColor=color(0, 1, 0) )#color de la celda

#inserto fila por fila en la base de datos
for i in range(1,len(wks_data)):
    query = f"BEGIN IF NOT EXISTS (SELECT * FROM ACTIVE.DBO.PERIODOS_TABLEROS WHERE PERIODO = '{wks_data[i][3]}') BEGIN INSERT INTO ACTIVE.DBO.PERIODOS_TABLEROS (IDCLIENTE, MES, AÑO, PERIODO, FINICIO, FFIN) VALUES {tuple(wks_data[i])} END END"
    try:
        cursor.execute(query)
        cnxn.commit()
        format_cell_range(worksheet, f'{i+1}', fmt)
        print('Insert Finish!')
    except: 
        print("No se pudo insertar")
cnxn.close()


Insert Finish!
Insert Finish!
Insert Finish!


## AUTOMATIZACION RELLENO REFIS


In [1]:
from utils.functions_ia import *
from gspread_formatting  import *

#me me conecto a la spreadsheet y agarro sus datos
key = "1Ed0M09j0yk2LCLU0sBmAUximavxcZoesNmDd1w25TC8"
sheet_name = "REFI (PAG_UNIF)"
gc = connect_spreadsheets()
sh = gc.open_by_key(key)
worksheet = sh.worksheet(sheet_name)

wks_data = worksheet.get_all_values() #es una lista de listas de cada fila del spreadsheet



In [6]:
worksheet.row(1).format()

AttributeError: 'Worksheet' object has no attribute 'row'

In [20]:
#me conecto a la base de datos
cnxn = connect_database() #genero la conexion con la db

#inserto los datos de la spreadsheet en la db
cursor = cnxn.cursor()

fmt = cellFormat(backgroundColor=color(0, 1, 0) )#color de la celda

#inserto fila por fila en la base de datos
for i in range(1,len(wks_data)):
    query = f"BEGIN IF NOT EXISTS (SELECT * FROM ACTIVE.DBO.PERIODOS_TABLEROS WHERE PERIODO = '{wks_data[i][3]}') BEGIN INSERT INTO ACTIVE.DBO.PERIODOS_TABLEROS (IDCLIENTE, MES, AÑO, PERIODO, FINICIO, FFIN) VALUES {tuple(wks_data[i])} END END"
    try:
        cursor.execute(query)
        cnxn.commit()
        format_cell_range(worksheet, f'{i+1}', fmt)
        print('Insert Finish!')
    except: 
        print("No se pudo insertar")
cnxn.close()


{'spreadsheetId': '1Ed0M09j0yk2LCLU0sBmAUximavxcZoesNmDd1w25TC8',
 'replies': [{}]}

In [26]:
import os


from utils.functions_ia import *

cnxn = connect_database()
query = "SELECT * FROM ACTIVE.DBO.TABLERO_LATE_COL_ENRIQUE_V2"
df = pd.read_sql(query, cnxn).dropna(how = "all")
df.columns = [col.upper() for col in df.columns]

key = "1u-TSpwfKI8G4SQPjx0TZUFPR2xl0GejeYhkuvMs2rWY"

query= "SELECT * FROM ACTIVE.DBO.EVOLUCION_ENRIQUECIMIENTO_VIEW"
df= pd.read_sql(query, cnxn).dropna(how='all')
df.columns = [col.upper() for col in df.columns]
print(df.columns)
sheet_name= "Evolucion"


Index(['IDMOROSO', 'IDCLIENTE', 'FASIGNACION', 'FECHA_ENVIO_ENRIQUECIMIENTO',
       'PROVEEDOR', 'ENRIQ_90DIAS', 'AÑO_MES'],
      dtype='object')


In [34]:
import numpy as np
np.shape(df)

(45950, 7)

In [36]:
type(worksheet)

gspread.models.Worksheet